In [2]:
# Import necessary libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, TensorDataset

# Parameters
BATCH_SIZE = 64
EPOCHS_CNN = 10
EPOCHS_PCA_NN = 25
LEARNING_RATE_CNN = 0.001
LEARNING_RATE_PCA_NN = 0.001
PCA_COMPONENTS = 150

# Load and preprocess the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# CNN Model (PyTorch) - Code 1
class CNNModel:
    def __init__(self):
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2), nn.Conv2d(32, 64, 3), nn.ReLU(),
            nn.MaxPool2d(2, 2), nn.Flatten(),
            nn.Linear(64 * 6 * 6, 128), nn.ReLU(),
            nn.Linear(128, 10)
        )
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE_CNN)

    def train(self, X_train, y_train):
        X_train_tensor = torch.tensor(X_train).unsqueeze(1)
        y_train_tensor = torch.tensor(y_train)
        train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=BATCH_SIZE, shuffle=True)
        
        self.model.train()
        for epoch in range(EPOCHS_CNN):
            total_loss = 0
            for batch_X, batch_y in train_loader:
                self.optimizer.zero_grad()
                loss = self.criterion(self.model(batch_X), batch_y)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            print(f"Epoch {epoch + 1}/{EPOCHS_CNN} - Loss: {total_loss / len(train_loader):.4f}")

    def evaluate(self, X, y):
        self.model.eval()
        with torch.no_grad():
            X_tensor, y_tensor = torch.tensor(X).unsqueeze(1), torch.tensor(y)
            accuracy = (torch.argmax(self.model(X_tensor), dim=1) == y_tensor).float().mean().item()
        return accuracy

# PCA-based Simple Neural Network (NumPy) - Code 2
class PCANeuralNetwork:
    def __init__(self):
        self.weights_input_hidden = np.random.randn(PCA_COMPONENTS, 256) * np.sqrt(2. / PCA_COMPONENTS)
        self.bias_hidden = np.zeros(256)
        self.weights_hidden_output = np.random.randn(256, 10) * np.sqrt(2. / 256)
        self.bias_output = np.zeros(10)

    def relu(self, x):
        return np.maximum(0, x)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        self.hidden_input = X @ self.weights_input_hidden + self.bias_hidden
        self.hidden_output = self.relu(self.hidden_input)
        return self.softmax(self.hidden_output @ self.weights_hidden_output + self.bias_output)

    def backward(self, X, y, output):
        error = output - y
        d_weights_hidden_output = self.hidden_output.T @ error
        hidden_error = (error @ self.weights_hidden_output.T) * (self.hidden_input > 0)
        d_weights_input_hidden = X.T @ hidden_error
        self.weights_hidden_output -= LEARNING_RATE_PCA_NN * d_weights_hidden_output
        self.bias_output -= LEARNING_RATE_PCA_NN * np.sum(error, axis=0)
        self.weights_input_hidden -= LEARNING_RATE_PCA_NN * d_weights_input_hidden
        self.bias_hidden -= LEARNING_RATE_PCA_NN * np.sum(hidden_error, axis=0)

    def compute_loss(self, y_true, y_pred):
        return -np.sum(np.log(y_pred[range(y_true.shape[0]), y_true.argmax(axis=1)] + 1e-8)) / y_true.shape[0]

    def train(self, X, y):
        for epoch in range(EPOCHS_PCA_NN):
            for i in range(0, X.shape[0], BATCH_SIZE):
                batch_X, batch_y = X[i:i + BATCH_SIZE], y[i:i + BATCH_SIZE]
                output = self.forward(batch_X)
                self.backward(batch_X, batch_y, output)
            train_accuracy = np.mean(np.argmax(self.forward(X), axis=1) == np.argmax(y, axis=1))
            print(f"Epoch {epoch + 1}/{EPOCHS_PCA_NN} - Training Accuracy: {train_accuracy * 100:.2f}%")

    def evaluate(self, X, y):
        predictions = self.forward(X)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy

# Preprocess data for both models
X_train_flat = X_train.reshape(-1, 28, 28)
X_test_flat = X_test.reshape(-1, 28, 28)

# Prepare labels for PCA-based model
y_train_encoded = np.eye(10)[y_train]
y_test_encoded = np.eye(10)[y_test]

# Instantiate and train both models
print("Training CNN Model (PyTorch)...")
cnn_model = CNNModel()
cnn_model.train(X_train_flat, y_train)
cnn_accuracy = cnn_model.evaluate(X_test_flat, y_test)

print("\nApplying PCA and Training Simple Neural Network (NumPy)...")
pca = PCA(n_components=PCA_COMPONENTS)
X_train_pca = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_pca = pca.transform(X_test.reshape(X_test.shape[0], -1))

pca_nn_model = PCANeuralNetwork()
pca_nn_model.train(X_train_pca, y_train_encoded)
pca_nn_accuracy = pca_nn_model.evaluate(X_test_pca, y_test_encoded)

# Benchmark results
print("\nBenchmark Results:")
print(f"CNN Model Test Accuracy: {cnn_accuracy * 100:.2f}%")
print(f"PCA + Simple NN Model Test Accuracy: {pca_nn_accuracy * 100:.2f}%")

Training CNN Model (PyTorch)...
Epoch 1/10 - Loss: 0.4857
Epoch 2/10 - Loss: 0.3166
Epoch 3/10 - Loss: 0.2631
Epoch 4/10 - Loss: 0.2320
Epoch 5/10 - Loss: 0.2067
Epoch 6/10 - Loss: 0.1858
Epoch 7/10 - Loss: 0.1649
Epoch 8/10 - Loss: 0.1497
Epoch 9/10 - Loss: 0.1350
Epoch 10/10 - Loss: 0.1203

Applying PCA and Training Simple Neural Network (NumPy)...
Epoch 1/25 - Training Accuracy: 85.26%
Epoch 2/25 - Training Accuracy: 86.84%
Epoch 3/25 - Training Accuracy: 87.67%
Epoch 4/25 - Training Accuracy: 88.20%
Epoch 5/25 - Training Accuracy: 88.67%
Epoch 6/25 - Training Accuracy: 89.11%
Epoch 7/25 - Training Accuracy: 89.45%
Epoch 8/25 - Training Accuracy: 89.76%
Epoch 9/25 - Training Accuracy: 89.99%
Epoch 10/25 - Training Accuracy: 90.20%
Epoch 11/25 - Training Accuracy: 90.42%
Epoch 12/25 - Training Accuracy: 90.63%
Epoch 13/25 - Training Accuracy: 90.85%
Epoch 14/25 - Training Accuracy: 91.01%
Epoch 15/25 - Training Accuracy: 91.21%
Epoch 16/25 - Training Accuracy: 91.34%
Epoch 17/25 - Tr